In [38]:
from threading import Thread

In [46]:
import time

def countdown(n):
    for i in range(n):
        print(n - i - 1, 'left')
        time.sleep(1)
        
t = Thread(target=countdown, args=(3, ))

In [47]:
t.start()
print('asdas')
time.sleep(0.5)
print('sdasd')

2asdas
 left
sdasd
1 left
0 left


> системные ресурсы разделяются - типа дескриптора i/o - от одного процесса 

> Как дождаться выполнения потока?

In [51]:
some_list = []

def appender(n):
    for i in range(n):
        some_list.append(n - i - 1)
        time.sleep(0.1)
        
t = Thread(target=appender, args=(5, ))

t.start()
t.join()
print(some_list)

[4, 3, 2, 1, 0]


_______-

In [57]:
some_list = []

def appender(n):
    for i in range(n):
        some_list.append(n - i - 1)
        time.sleep(0.2)
        
t = Thread(target=appender, args=(5, ))
t1 = Thread(target=appender, args=(5, ))

In [58]:
%%time

t.start()
t1.start()
t.join()
t1.join()

print(some_list)

[4, 4, 3, 3, 2, 2, 1, 1, 0, 0]
CPU times: user 0 ns, sys: 2.57 ms, total: 2.57 ms
Wall time: 1 s


_______

In [70]:
def sleeper(n):
    for i in range(n):
        time.sleep(2.2)
        
t = Thread(target=sleeper, args=(5, ), daemon=True)

In [71]:
t.start()
t.isAlive()

True

In [73]:
t.daemon

True

In [74]:
t.join() 

In [75]:
t.isAlive()

False

> при завершении работы интерпретатора потоки демоны должны автоматически быть уничтожены. 

> для обычных тредов - это не так

In [123]:
from multiprocessing import Process

In [128]:
some_list = []

def appender(n):
    print(type(some_list))
    for i in range(n):
        some_list.append(n - i - 1)
        time.sleep(0.2)
    return some_list    
    
t = Process(target=appender, args=(5, ))
t1 = Process(target=appender, args=(5, ))

In [129]:
t.start()
t1.start()

t.join()
t1.join()

<class 'type'>
<class 'type'>


In [130]:
t.is_alive()

False

In [131]:
some_list

[]

Круто. Умеем создавать свои потоки и процессы.

In [136]:
some_list = []

def appender(n):
    for i in range(n):
        some_list.append(n - i - 1)
#         time.sleep(0.001)
        
t = Thread(target=appender, args=(250, ))
t1 = Thread(target=appender, args=(250, ))

t.start()
t1.start()
t.join()
t1.join()

print(some_list)

[249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 3

##### GIL

> GIL (global interpreter lock)—это мьютекс, который гарантирует, что в каждый момент времени только один поток имеет доступ к внутреннему состоянию интерпретатора

GIL - зачем? 

- чтобы избежать состояния гонки
- чтобы избежать порчи данных (один пришел, добавил, другой пришел считал до того как добавил и положил, первое пропало)
- 

У питона под капотом GIL, но GIL отпускается когда идут какие либо системные вызовы (работа с сокетами, i/o и тд.)

![](https://cdn-images-1.medium.com/max/409/1*_4B2PKsJn9pUz3jbTnBnYw.png)

Пойдем дальше, больше потоков, больше процессов! Будем быстрее и осторожнее

##### Пример с concurrent.futures

In [83]:
import concurrent.futures

In [ ]:
executor.map (функция, итерируемые объекты)
executor.submit (функция, аргументы)
executor.shutdown (wait=True)

In [ ]:
def summarize(n):
    s = 0
    for i in range(n):
        s += i
    time.sleep(1)
    return s

In [85]:
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    f = executor.submit(summarize, 100)
    f1 = executor.submit(summarize, 144)
    
    print(f.done(), f.running())
    
    print(f.result())  # <- блокирует
    
    print(f1.done(), f1.running())
    

f1.result()

False True
4950
True False


10296

In [86]:
with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
    f = executor.submit(summarize, 100)
    f1 = executor.submit(summarize, 144)
    
    print(f.done(), f.running())
    
    print(f.result())  # <- блокирует
    
    print(f1.done(), f1.running())
    

f1.result()

False True
4950
True False


10296

___________-

##### Что с шаренными ресурсами для процесса и потока?

In [104]:
s = []

In [113]:
def appendaizer(n):
    print(type(s))
    for i in range(n):
        s.append(i)
    time.sleep(1)
    return s

In [114]:
with concurrent.futures.ProcessPoolExecutor(max_workers=1) as executor:
    f = executor.submit(appendaizer, 100)
    
    print(f.done(), f.running())
    print(f.result())  # <- блокирует

<class 'list'>
False True
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [115]:
print(s)

[]

In [138]:
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    f = executor.submit(appendaizer, 100)
    
    print(f.done(), f.running())
    
    print(f.result())  # <- блокирует

<class 'list'>
False True
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [140]:
print(s)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


__________________________--

##### Жиза

In [87]:
import time
import requests
from bs4 import BeautifulSoup
from functools import lru_cache
import numpy as np

proxies_list = [{'http':'http://'+i} for i in [
                '67.149.217.254:10200',
                '64.20.74.24:45554','62.37.237.101:8080',
                '180.234.206.77:8080',
                '78.11.85.13:8080','109.188.81.101:8080',
                '139.59.17.113:8080','191.179.147.46:11421']]

# for i in range(len(proxies_list)):
#     time.sleep(np.random.random())
#     assert 200 == requests.get('https://yandex.ru', 
#                                proxies=proxies_list[i]).status_code
# print ('all_proxies_are_worked!\n\t----\n')

In [88]:
headers = {
    'User-Agent': 'Mozila 5/0',
}

@lru_cache(maxsize=100000)
def get_product_info(idx):
    response = requests.get('https://pandao.ru/product/'+idx, 
                            headers=headers, 
                            proxies=proxies_list[np.random.randint(0, len(proxies_list))])
    #time.sleep(0.2)
    if not response.ok:
        return {'idx':idx, 'title':0, 'price':0, 'old_price':0, 
            'rating':0, 'orders':0}
    
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.find('div', {'class':"product-title"})
    if title:
        title = ' '.join(title.text.split())
    else:
        title = 'None'

    price = soup.find('p', {'class':"current-price"})
    if price:
        price = float(price.attrs['content'])
    else:
        price = 0

    rating = soup.find('span', {'class':"sale-rating"})
    if rating:
        rating = float(rating.text.split(' ')[0])
    else:
        rating = 0
        
    return {'idx':idx, 'title':title, 'price':price, 
            'rating':rating }

In [89]:
prd_ids = ['0ddd2f40-39cf-4288-9c4d-1b32f2ddd142', '97154181-8921-48be-9397-35c2ff5f5111',
'5a0a0808-c432-406b-ba36-382a797ae74a', '2f21e653-0fe4-4b35-867b-29eefe9a1b39',
'0dcad2bc-226d-4e37-b602-754ac564ea50', '017a2150-d5a0-413b-9be3-0d59bdddc895',
'464a9254-e7f4-4bde-8f09-c356e59228be', 'dd507101-c239-46ef-86c0-7d8cdb598374',
'41ccd8d7-1514-4534-9958-beb7a1ee9797', '41ccd8d7-1514-4534-9958-beb7a1ee9797',
'8a3ec31f-8b80-45a1-aab0-51b6425005f7', 'b80043b6-67f5-448c-8565-1a0d6c8b3449',
'b902f630-68c0-4be5-93ec-d7d1d322c97e', '5f98612e-d999-426c-a74d-d3a2439e5ded',
'2ea3a39f-7f96-4e45-b04a-7f7349100f6b', '65058295-d8ca-4d36-871b-e1a50247656e',
'95414baf-0203-4c6a-9b90-f3bb9e0c3bae',
'94bf017f-3a3c-48f2-95cf-972ab92b5c0f', 'c094fe15-b54e-445d-ba1b-a2897e9710d4',
'126d746f-0458-4874-85e3-47d72e9bd261',
'592d4b46-e7b5-4a06-86f8-0fda04047918', '3e5348a1-add0-410e-9ef6-5f919ca65a3f',
'5600652c-ce34-41c1-9f57-1066eb8a0e6c', 'e4d0ea4f-fc51-4af4-b996-4479d4b06cdd',
'2b3c649b-b7dc-462f-9b23-cf92d7d373ba',
'cd5b75ff-010f-416f-846a-9886d935a2fc', '88c7d55b-6dff-4878-8f7b-c82a9302ea17',
'65e2be8a-bd34-4e8b-9cf0-774d7e0e99a0',
'bde6300a-b268-4724-ac2d-88e439acabf0', '9566efbd-b94f-42a6-b13f-eecb51a54a51',
'e16c4a79-79e2-46ea-a0d0-9cfb863d110a',
'43c43bb6-d9fd-4e8b-ac6c-f572cbc944d4', 'eabd6d3a-0e12-4c01-862d-8eb1e0ee67c5',
'94c58080-3fcd-408a-8a2e-10297f61060e', 'a9fb97f1-8534-410d-bd1a-0f8341311c48',
'e8437e9b-96b5-45fc-b2d7-27d54abfbd4e', '9af479fa-9904-419d-9289-a9f7a254b5f6',
'3568d795-e02d-42fb-a85b-a488e01445cc',
'43b92387-d118-4fc5-871e-6615417b1d79', 'fc2a66f4-550a-4944-b796-cc4c15ac7201',
'96b701c9-a14f-4fa0-8a46-99ac62418598',
'b158cc8b-279e-45c9-9b06-20ec25e1cf42', '7687608c-c3d0-48a5-893e-7a2f5a277578',
'906c782b-119f-4398-bac6-9969b4c117ab',
'8a708ccd-6a51-4712-bec6-912fcc2bc2d3', 'b4851d65-1e38-473c-a190-2e862f4cb8cd',
'd456cfb8-c2da-4f89-abcc-16df65bcfb18', '51455200-78a7-4353-8f08-c90b4f149165',
'c0e41577-42c4-4402-a89c-5418da501b8c', '90516e9b-ace8-43bf-8eca-f8df95b1f03a',
'027f8d2f-dc8f-4dff-a5e8-87a5d3cfeb69', '14bc4955-48b1-43d3-9dc7-b18332367254',
'17a22405-34ec-45dc-b7a5-35034b8f9346',
'3d954a88-1534-499e-9b01-d82cca8afda3', '3eacaa93-ab98-4165-bec8-5ae96adb4df8',
'952570e0-19c1-44af-bd6d-aa7d546986c8',
'a27f0148-d5e6-4457-ad21-1d257f270967' , 'b82844af-d28d-4f33-9038-c9fd27bc5c24',
'0e873862-8750-4604-9dd8-b718b1391532',
'cc362ad5-e232-4727-87d0-1c571f1eefe1', '9c237e0b-10ac-48c1-a7fe-7bb6f72ce927',
'e87632c9-6764-4ee5-862f-0b84b7ce2914', '14e38f1a-1d95-4238-8381-ad3c3fc2d31e',
'4c6bc3d5-a389-4f57-9d89-f61d4eacc77b',
'7933813d-f6a0-4352-ba4f-824bc98680a8', '32781d36-806b-45eb-bb2d-4bca80d8b6f2',
'72cd2efc-51d9-4a45-a710-7fc62359a4d9', '4c8ac5d2-0307-462a-aaac-ba6ffe562a55',
'81ef7c5e-d936-4f44-96d8-3a6cb87be0c4', '8f92e4b3-4d09-46f3-a24b-3da800c64c77',
'380d96de-e974-4141-af46-2f765e5fc984', '6fd9abec-7ef8-4842-b24d-c06b4e60bc59', 
'45253494-0c26-4a21-ae11-569343c55ade',
'fb524119-dd0c-4575-ad67-31f116c0ad5c', '0f32e154-9063-4e31-a6e6-2a5f41a8d16c',
'06839e0e-80d7-490a-a1d1-706cb0f65837', '4fd83c9f-cd16-477c-8054-ff17b5c225c3',
'c14c3277-f011-4da1-9d64-56ce89d6e23c',
'9534385e-2373-48f4-9c6a-9ef1c3b02e09', '488d58b0-faa4-483a-ae4d-d541dc436483', 
'740b5673-11b1-43a8-b2cc-558e5bb9705d', '94c58080-3fcd-408a-8a2e-10297f61060e',
'36da7efb-4f78-4325-9f4b-fc9ea2f76e3e',
'f38acc05-d3d9-43a6-a310-e37e35a06258', '5f98612e-d999-426c-a74d-d3a2439e5ded',
'63baf2b4-89ac-4667-ba39-e45596bdc7fc',
'95414baf-0203-4c6a-9b90-f3bb9e0c3bae', '340c64be-c30b-4be3-83dd-e675c7ed533b',
'39cca812-e5e7-4c76-a521-def0292fb184', '6fd9abec-7ef8-4842-b24d-c06b4e60bc59',
'7bbe3f2c-d5ef-4f23-878c-4c9abec6c75e', '1c39cf24-1ae1-445f-b9e7-233cb102e9ce',
'786860b0-dfb6-4a20-b2b0-f40f69088508', '9af479fa-9904-419d-9289-a9f7a254b5f6', 
'22fba623-c746-45af-9d94-7d78f73ef031', 'a08a7522-3dfb-49cd-a682-b3c32d5538d5',
'73b8ff53-288b-46f8-85a8-593e198ba491', '85a89ff0-a438-40fe-aa59-d47ac5353b5d',
'955eedc1-4211-4d42-976c-0f5149a29d2a',
'349982e9-ad32-4e27-8c68-c4e8e4e4491b', '9798f731-b4af-45f4-88aa-5798d1e84c27',
'19b9e842-7756-4a71-a01e-081c3819aa17',
'756ac7ba-88ce-476e-a4ba-24a175273aeb', '49b4589d-6421-4852-8d88-4eb6cb317064',
'455f22e7-732b-478b-a412-822436c790a9'
          ]

In [90]:
len(prd_ids)

105

In [145]:
# do not forget for tests !
get_product_info.cache_clear()

In [146]:
result = {}
time_begin = datetime.datetime.now()

for i, idx in enumerate(prd_ids):
    data = get_product_info(idx)
    result[data['idx']] = data
    if i % 10 == 0:
        print ('computed %d lines in %s' % (i, str(datetime.datetime.now() - time_begin)[:11]))
        
time_end = datetime.datetime.now()
print('whole time = ', time_end-time_begin)

computed 0 lines in 0:00:00.461
computed 10 lines in 0:00:04.489
computed 20 lines in 0:00:09.196
computed 30 lines in 0:00:13.415
computed 40 lines in 0:00:17.827
computed 50 lines in 0:00:22.352
computed 60 lines in 0:00:26.572
computed 70 lines in 0:00:30.883
computed 80 lines in 0:00:36.821
computed 90 lines in 0:00:39.916
computed 100 lines in 0:00:44.648
whole time =  0:00:46.347036


In [147]:
import pandas as pd
pd.DataFrame(result).T.head()

,idx,price,rating,title
0ddd2f40-39cf-4288-9c4d-1b32f2ddd142,0ddd2f40-39cf-4288-9c4d-1b32f2ddd142,84,2377,Спрей для защиты экрана телефона
97154181-8921-48be-9397-35c2ff5f5111,97154181-8921-48be-9397-35c2ff5f5111,365,996,Воздушный шар для воды
5a0a0808-c432-406b-ba36-382a797ae74a,5a0a0808-c432-406b-ba36-382a797ae74a,272,3076,Настенный держатель для душа
2f21e653-0fe4-4b35-867b-29eefe9a1b39,2f21e653-0fe4-4b35-867b-29eefe9a1b39,44,9797,"Светодиодная гирлянда, работающая от батареек"
0dcad2bc-226d-4e37-b602-754ac564ea50,0dcad2bc-226d-4e37-b602-754ac564ea50,107,49131,Женские наручные кварцевые часы с ремешком из ...


In [151]:
# do not forget for tests !
get_product_info.cache_clear()

In [152]:
import datetime
import concurrent.futures

CONNECTIONS = 4
time_begin = datetime.datetime.now()
result = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    
    dicts_with_result = {
        executor.submit(get_product_info, idx): idx for idx in prd_ids}
    
    for i, future in enumerate(concurrent.futures.as_completed(dicts_with_result)):
        if i % 10 == 0:
            print ('computed %d lines in %s' % (i, str(datetime.datetime.now() - time_begin)[:11]))
        try:
            data = future.result()
            result[data['idx']] = data
        except Exception as exc:
            print (exc, i, future)
        finally:
            pass
        
time_end = datetime.datetime.now()

print('whole time = ', time_end-time_begin)

computed 0 lines in 0:00:00.583
computed 10 lines in 0:00:02.156
computed 20 lines in 0:00:03.700
computed 30 lines in 0:00:05.213
computed 40 lines in 0:00:06.656
computed 50 lines in 0:00:08.707
computed 60 lines in 0:00:10.006
computed 70 lines in 0:00:11.842
computed 80 lines in 0:00:13.279
computed 90 lines in 0:00:13.897
computed 100 lines in 0:00:15.320
whole time =  0:00:15.791971


In [153]:
import pandas as pd
pd.DataFrame(result).T.head()

,idx,price,rating,title
0ddd2f40-39cf-4288-9c4d-1b32f2ddd142,0ddd2f40-39cf-4288-9c4d-1b32f2ddd142,84,2377,Спрей для защиты экрана телефона
97154181-8921-48be-9397-35c2ff5f5111,97154181-8921-48be-9397-35c2ff5f5111,365,996,Воздушный шар для воды
5a0a0808-c432-406b-ba36-382a797ae74a,5a0a0808-c432-406b-ba36-382a797ae74a,272,3076,Настенный держатель для душа
2f21e653-0fe4-4b35-867b-29eefe9a1b39,2f21e653-0fe4-4b35-867b-29eefe9a1b39,44,9797,"Светодиодная гирлянда, работающая от батареек"
017a2150-d5a0-413b-9be3-0d59bdddc895,017a2150-d5a0-413b-9be3-0d59bdddc895,226,6092,Силиконовая форма для выпечки блинов


In [92]:
# do not forget for tests !
get_product_info.cache_clear()

In [93]:
CONNECTIONS = 10
time_begin = datetime.datetime.now()
result = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    
    dicts_with_result = {executor.submit(get_product_info, idx): idx 
                   for idx in prd_ids}
    
    for i, future in enumerate(concurrent.futures.as_completed(dicts_with_result)):
        if i % 10 == 0:
            print ('computed %d lines in %s' % (i, str(datetime.datetime.now() - time_begin)[:11]))
        try:
            data = future.result()
            result[data['idx']] = data
        except Exception as exc:
            print (exc, i, future)
        finally:
            pass
        
time_end = datetime.datetime.now()

print('whole time = ', time_end-time_begin)

computed 0 lines in 0:00:00.693
computed 10 lines in 0:00:01.957
computed 20 lines in 0:00:03.496
computed 30 lines in 0:00:04.332
computed 40 lines in 0:00:05.906
computed 50 lines in 0:00:07.271
computed 60 lines in 0:00:08.244
computed 70 lines in 0:00:09.253
computed 80 lines in 0:00:09.895
computed 90 lines in 0:00:10.868
computed 100 lines in 0:00:11.798
whole time =  0:00:11.990581


In [94]:
import pandas as pd
pd.DataFrame(result).T.head()

In [154]:
# do not forget for tests !
get_product_info.cache_clear()

In [155]:
CONNECTIONS = 10
time_begin = datetime.datetime.now()
result = {}

with concurrent.futures.ProcessPoolExecutor(max_workers=CONNECTIONS) as executor:
    
    dicts_with_result = {executor.submit(get_product_info, idx): idx 
                   for idx in prd_ids}
    
    for i, future in enumerate(concurrent.futures.as_completed(dicts_with_result)):
        if i % 10 == 0:
            print ('computed %d lines in %s' % (i, str(datetime.datetime.now() - time_begin)[:11]))
        try:
            data = future.result()
            result[data['idx']] = data
        except Exception as exc:
            print (exc, i, future)
        finally:
            pass
        
time_end = datetime.datetime.now()

print('whole time = ', time_end-time_begin)

computed 0 lines in 0:00:00.671
computed 10 lines in 0:00:01.886
computed 20 lines in 0:00:02.668
computed 30 lines in 0:00:03.309
computed 40 lines in 0:00:03.937
computed 50 lines in 0:00:04.586
computed 60 lines in 0:00:05.224
computed 70 lines in 0:00:05.812
computed 80 lines in 0:00:06.348
computed 90 lines in 0:00:06.925
computed 100 lines in 0:00:07.389
whole time =  0:00:07.815100


In [156]:
pd.DataFrame(result).T.head()

,idx,price,rating,title
0ddd2f40-39cf-4288-9c4d-1b32f2ddd142,0ddd2f40-39cf-4288-9c4d-1b32f2ddd142,84,2377,Спрей для защиты экрана телефона
017a2150-d5a0-413b-9be3-0d59bdddc895,017a2150-d5a0-413b-9be3-0d59bdddc895,226,6092,Силиконовая форма для выпечки блинов
5a0a0808-c432-406b-ba36-382a797ae74a,5a0a0808-c432-406b-ba36-382a797ae74a,272,3076,Настенный держатель для душа
97154181-8921-48be-9397-35c2ff5f5111,97154181-8921-48be-9397-35c2ff5f5111,365,996,Воздушный шар для воды
41ccd8d7-1514-4534-9958-beb7a1ee9797,41ccd8d7-1514-4534-9958-beb7a1ee9797,131,82114,Беспроводные Bluetooth-наушники для мобильного...


In [157]:
# do not forget for tests !
get_product_info.cache_clear()

In [158]:
CONNECTIONS = 20
time_begin = datetime.datetime.now()
result = {}

with concurrent.futures.ProcessPoolExecutor(max_workers=CONNECTIONS) as executor:
    
    dicts_with_result = {executor.submit(get_product_info, idx): idx 
                   for idx in prd_ids}
    
    for i, future in enumerate(concurrent.futures.as_completed(dicts_with_result)):
        if i % 10 == 0:
            print ('computed %d lines in %s' % (i, str(datetime.datetime.now() - time_begin)[:11]))
        try:
            data = future.result()
            result[data['idx']] = data
        except Exception as exc:
            print (exc, i, future)
        finally:
            pass
        
time_end = datetime.datetime.now()

print('whole time = ', time_end-time_begin)

computed 0 lines in 0:00:02.950
computed 10 lines in 0:00:05.312
computed 20 lines in 0:00:06.745
computed 30 lines in 0:00:06.925
computed 40 lines in 0:00:11.869
computed 50 lines in 0:00:12.718
computed 60 lines in 0:00:12.938
computed 70 lines in 0:00:13.738
computed 80 lines in 0:00:14.009
computed 90 lines in 0:00:14.618
computed 100 lines in 0:00:14.746
whole time =  0:00:15.446448


In [159]:
pd.DataFrame(result).T.head()

,idx,price,rating,title
41ccd8d7-1514-4534-9958-beb7a1ee9797,41ccd8d7-1514-4534-9958-beb7a1ee9797,131,82114,Беспроводные Bluetooth-наушники для мобильного...
2ea3a39f-7f96-4e45-b04a-7f7349100f6b,2ea3a39f-7f96-4e45-b04a-7f7349100f6b,34,27148,Подставка для мобильного телефона
3e5348a1-add0-410e-9ef6-5f919ca65a3f,3e5348a1-add0-410e-9ef6-5f919ca65a3f,35,734,Прозрачный силиконовый защитный чехол для iPho...
592d4b46-e7b5-4a06-86f8-0fda04047918,592d4b46-e7b5-4a06-86f8-0fda04047918,81,2103,Кабель usb с магнитным разъемом Type-C / Micro...
5600652c-ce34-41c1-9f57-1066eb8a0e6c,5600652c-ce34-41c1-9f57-1066eb8a0e6c,58,448,"Прозрачный чехол для iPhone 5, 5s, SE, 6, 6S, ..."


> много процессов - плохо!

___________-

Ск:чно. Давайте зарешаем мльную задачу - классификация категории объявления - для чего может понадобиться?

In [162]:
import re
import pymorphy2
from functools import lru_cache

In [163]:
import numpy as np
import pandas as pd

In [180]:
from multiprocessing import Pool

In [208]:
data = pd.read_csv('data/train.csv')

In [166]:
data.head()

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [167]:
morph = pymorphy2.MorphAnalyzer()

@lru_cache(maxsize=100000) # с кешом 14с против 1min 16s
def get_normal_form (i):
    return morph.normal_forms(i)[0]

def normalize_text(x):
    x = x[:200] 
    return ' '.join([get_normal_form(i) for i in re.findall('\w+', x)])

TODO

> попробуйте дома поиграться с тем какой прирост дает кеш, какой параллелирование.


In [168]:
get_normal_form('стулья')

'стул'

In [169]:
normalize_text('Стулья из прессованной кожи 23432 руб.')

'стул из прессовать кожа 23432 рубль'

In [170]:
data.head().title.apply(lambda x: normalize_text(x))

0                                           картина
1                           стул из прессовать кожа
2                                домашний минь баня
3    эксклюзивный коллекция книга трансаэро подарок
4                                      ноутбук aser
Name: title, dtype: object

In [171]:
%%time

_ = data.title.apply(lambda x: normalize_text(x))

CPU times: user 39.9 s, sys: 136 ms, total: 40 s
Wall time: 40.1 s


> ? как ускорить?

In [172]:
get_normal_form.cache_clear() # чтобы быть честными!

In [35]:
%%time

_ = data.head(100000).title.apply(lambda x: normalize_text(x))

CPU times: user 12.9 s, sys: 48 ms, total: 12.9 s
Wall time: 12.9 s


In [173]:
get_normal_form.cache_clear()

In [174]:
%%time
with Pool(processes=1) as pool:
    _ = pool.map(normalize_text, data.head(100000).title)
    pool.terminate()

CPU times: user 109 ms, sys: 96 ms, total: 205 ms
Wall time: 12.7 s


In [175]:
get_normal_form.cache_clear()

In [176]:
%%time
with Pool(processes=4) as pool:
    _ = pool.map(normalize_text, data.head(100000).title)
    pool.terminate()

CPU times: user 92.4 ms, sys: 192 ms, total: 285 ms
Wall time: 9.49 s


In [177]:
get_normal_form.cache_clear()

In [178]:
%%time
with Pool(processes=20) as pool:
    _ = pool.map(normalize_text, data.head(100000).title)
    pool.terminate()

CPU times: user 141 ms, sys: 681 ms, total: 822 ms
Wall time: 15.1 s


In [195]:
get_normal_form.cache_clear()

##### Советы по оптимизации кода!
https://scikit-learn.org/stable/developers/performance.html

In [1]:
from sklearn.externals.joblib import Parallel, delayed

# https://joblib.readthedocs.io/en/latest/

In [214]:
# parallel_proc = Parallel(n_jobs=10, backend='threading', verbose=1)

parallel_proc = Parallel(n_jobs=-1, backend='multiprocessing', verbose=1)
our_preprocessing_func = delayed(normalize_text)

In [215]:
%%time

res = parallel_proc( our_preprocessing_func(query) 
                    for query in data.head(100000).title.values)

[Parallel(n_jobs=-1)]: Done 12180 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 77880 tasks      | elapsed:    7.9s


CPU times: user 671 ms, sys: 172 ms, total: 843 ms
Wall time: 9.86 s


[Parallel(n_jobs=-1)]: Done 100000 out of 100000 | elapsed:    9.3s finished


In [216]:
%%time

parallel_proc = Parallel(n_jobs=-1, backend='threading', verbose=1)
res = parallel_proc( our_preprocessing_func(query) for query in data.head(100000).title.values)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 11242 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 12792 tasks      | elapsed:   

CPU times: user 22.3 s, sys: 882 ms, total: 23.2 s
Wall time: 22.8 s


[Parallel(n_jobs=-1)]: Done 100000 out of 100000 | elapsed:   22.7s finished


In [217]:
%%time

get_normal_form.cache_clear()
CONNECTIONS = 4
res = {}

time_begin = datetime.datetime.now()

with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    
    dicts_with_result = {executor.submit(normalize_text, idx): idx for idx in data.head(100000).title}
    
    for i, future in enumerate(concurrent.futures.as_completed(dicts_with_result)):
        data_ = future.result()
        #res[i] = data_
        
        if i % 20000 == 0:
            print ('computed %d lines in %s' % (i, str(datetime.datetime.now() - time_begin)[:11]))

computed 0 lines in 0:00:20.676
computed 20000 lines in 0:00:20.726
computed 40000 lines in 0:00:20.774
computed 60000 lines in 0:00:20.822
computed 80000 lines in 0:00:20.870
CPU times: user 20.8 s, sys: 986 ms, total: 21.8 s
Wall time: 21.1 s


In [218]:
%%time

get_normal_form.cache_clear()
CONNECTIONS = 4
res = {}

time_begin = datetime.datetime.now()

with concurrent.futures.ProcessPoolExecutor(max_workers=CONNECTIONS) as executor:
    
    dicts_with_result = {executor.submit(normalize_text, idx): idx for idx in data.head(100000).title}
    for i, future in enumerate(concurrent.futures.as_completed(dicts_with_result)):
        
        data_ = future.result()
        # res[i] = data_
        
        if i % 20000 == 0:
            print ('computed %d lines in %s' % (i, str(datetime.datetime.now() - time_begin)[:11]))

computed 0 lines in 0:00:10.082
computed 20000 lines in 0:00:14.353
computed 40000 lines in 0:00:18.647
computed 60000 lines in 0:00:23.112
computed 80000 lines in 0:00:28.157
CPU times: user 25.8 s, sys: 4.43 s, total: 30.2 s
Wall time: 33.3 s


Ответ:

In [9]:
%%time
with Pool(processes=4) as pool:
    data['title_parse'] = pool.map(normalize_text, data.title)
    pool.terminate()

CPU times: user 539 ms, sys: 163 ms, total: 702 ms
Wall time: 26.3 s


In [10]:
%%time
with Pool(processes=4) as pool:
    data['description_parse'] = pool.map(normalize_text, data.description)
    pool.terminate()

CPU times: user 1.75 s, sys: 563 ms, total: 2.31 s
Wall time: 1min 26s


In [11]:
import scipy.sparse as sp

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

/home/amir/venv/ds_venv3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
vectorizer = CountVectorizer(max_features=1000, binary=True)
title_features = vectorizer.fit_transform(data.title_parse)

vectorizer2 = CountVectorizer(max_features=1000, binary=True)
description_features = vectorizer.fit_transform(data.description_parse)

features = sp.hstack([title_features, description_features,
                      data[['price']] ])

In [13]:
vectorizer

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

##### https://docs.scipy.org/doc/scipy/reference/sparse.html

In [14]:
Xtrain, Xval, ytrain, yval = train_test_split(features, data.category_id, 
                                              random_state=241, test_size=0.33)

In [15]:
Xtrain.toarray()

<327976x2001 sparse matrix of type '<class 'numpy.float64'>'
	with 4430973 stored elements in Compressed Sparse Row format>

In [16]:
%%time

clf = LogisticRegression()
clf.fit(Xtrain, ytrain)
y_pred = clf.predict(Xval)

CPU times: user 12min 20s, sys: 366 ms, total: 12min 20s
Wall time: 12min 21s


In [17]:
accuracy_score(yval, y_pred)

0.6691366278529909

лучше? 

> топ 10к, 20к или больше частотных слов

> почистить от шума (в, из, на, с, рублей, ...)

> выделять важные слова для отдельных категорий

> более сложные модели типа XGBoost or lgbm or catboost

In [ ]:
from multiprocessing import Pool

from subprocess import Popen, PIPE 
# run, call, check_output,  <-- хотим из питона что нибудь вызвать.

In [ ]:
def run_docker(cmd, container_name):  
    try:
        proc = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
        _ = proc.wait()
        if _!=0:
            #print(proc.communicate())
            print(proc.stdout())
            return False
    except Exception as e:
        print (proc.communicate())
        print (e)
        proc_kill = Popen('docker stop '+ container_name, shell=True, stdout=PIPE)
        proc_kill.wait()
        proc_kill = Popen('docker rm ' + container_name, shell=True, stdout=PIPE)
        proc_kill.wait()
        proc.kill()
        return False
    return True

DOCKER_NAME = 'brain_pipeline/frsrf_minc_fsl'

def bet_brain_mask(docker_container_name, path_to_sourse, file_name):
    return '''docker run --rm \
    --name {1} \
    -v {2}:/input \
    {0} \
    bash -c ". /etc/fsl/5.0/fsl.sh \
    && bet /input/{3} /input/brainmask_{3} -m -n -f .4\
    " '''.format(DOCKER_NAME, # 0
                 docker_container_name, # 1
                 path_to_sourse, # 2
                 file_name # 3
                 )

def run_for_patetnt(id_pat):
    cmd = bet_brain_mask()
    run_docker(cmd)


with Pool(5) as pool:
    result = pool.map(run_for_patetnt, tmp_arr[:20])

Пример работы с очередями

In [ ]:
from multiprocessing import Process, Manager

In [ ]:
input_queue, output_queue = Manager().Queue(), Manager().Queue()

gr = wrk_hf.groupby("Query")

for group in tqdm(gr.groups):
    input_queue.put(gr.get_group(group))
    
jobs = []

for i in tqdm(range(10)):
    job = Process(target=dedublication, args=(input_queue, output_queue,))
    job.start()
    jobs.append(job)
    
print("Start joining")
for job in jobs:
    job.join()
    
print("Start agreg")
df_j = []
while not output_queue.empty():
    df_j.append(output_queue.get(False))
